In [2]:
import time
from datetime import datetime as dt

from google.cloud import bigtable
from google.cloud.bigtable import column_family, row_filters
from google.cloud.bigtable.row_set import RowSet
import google.cloud.bigtable.row_filters as row_filters
import pandas as pd



In [3]:
INSTANCE_ID = 'bigtable-1'
TABLE_ID = 'plant1generation_{}'.format(time.time())

In [4]:
TABLE_ID

'plant1generation_1649637459.2678251'

### Utils functions

In [5]:
def print_row(row):
    print("Reading data for {}:".format(row.row_key.decode('utf-8')))
    for cf, cols in sorted(row.cells.items()):
        print("Column Family {}".format(cf))
        for col, cells in sorted(cols.items()):
            for cell in cells:
                labels = " [{}]".format(",".join(cell.labels)) \
                    if len(cell.labels) else ""
                print(
                    "\t{}: {} @{}{}".format(col.decode('utf-8'),
                                            cell.value.decode('utf-8'),
                                            cell.timestamp, labels))
    print("")

### Connecting to BigTable

In [6]:
client = bigtable.Client.from_service_account_json('./json/unbosque-service-account.json', admin = True)

In [7]:
print('Connecting to database instance')
instance = client.instance(INSTANCE_ID)

Connecting to database instance


### Creating a table

In [8]:
print('Creating the {} table'.format(TABLE_ID))
table = instance.table(TABLE_ID)

if not table.exists():
    table.create()
else:
    print("ERROR: Table {} already exists".format(TABLE_ID))

Creating the plant1generation_1649637459.2678251 table


In [9]:
print('Creating all family columns')

plant1ID = 'Power_Generation'
plant1_cf = table.column_family(plant1ID)
plant1_cf.create()


Creating all family columns


In [11]:
p1_generation = pd.read_csv("../data/Plant_1_Generation_Data.csv")
p1_json = p1_generation.to_dict(orient='records')
p1_temperature = pd.read_csv("../data/Plant_1_Weather_Sensor_Data.csv")
p1_jsonT = p1_temperature.to_dict(orient='records')
p2_generation = pd.read_csv("../data/Plant_2_Generation_Data.csv")
p2_json = p2_generation.to_dict(orient='records')
p2_temperature = pd.read_csv("../data/Plant_2_Weather_Sensor_Data.csv")
p2_jsonT = p2_temperature.to_dict(orient='records')